In [1]:
import sys,os
sys.path.append('../')
from smodels.tools import runtime
# Define your model (list of BSM particles)
runtime.modelFile = 'smodels.share.models.mssm'
# runtime.modelFile = 'mssmQNumbers.slha'

from smodels.theory import decomposer
from smodels.tools.physicsUnits import fb, GeV, TeV
from smodels.theory.theoryPrediction import theoryPredictionsFor
from smodels.experiment.databaseObj import Database
from smodels.tools import coverage
from smodels.tools.theoryPredictionsCombiner import TheoryPredictionsCombiner
from smodels.tools.smodelsLogging import setLogLevel
from smodels.particlesLoader import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.theory.model import Model
setLogLevel("info")

# Set the path to the database
database = Database(os.path.expanduser("~/smodels-database"))

INFO in databaseObj.loadBinaryFile() in 503: loading binary db file /home/lessa/smodels-database/db31.pcl format version 214
INFO in databaseObj.loadBinaryFile() in 510: Loaded database from /home/lessa/smodels-database/db31.pcl in 2.0 secs.


### Load model

In [2]:
slhafile = '../inputFiles/slha/simplyGluino.slha'
model = Model(BSMparticles=BSMList, SMparticles=SMList)
model.updateParticles(inputFile=slhafile)


INFO in model.updateParticles() in 383: Loaded 62 BSM particles


### Decompose

In [3]:
# Set main options for decomposition
sigmacut = 0.005*fb
mingap = 5.*GeV

# Decompose model
topDict = decomposer.decompose(model, sigmacut,
                               massCompress=True, invisibleCompress=True,
                               minmassgap=mingap)

#### Access basic information from decomposition, using the topology list and topology objects:

In [4]:

print("\n Decomposition Results: ")
print("\t  Total number of topologies: %i " % len(topDict))
nel = len(topDict.getElements())
print("\t  Total number of elements = %i " % nel)
# Print information about the m-th topology:
m = 0
if len(topDict) > m:
    cName = sorted(topDict.keys())[m]
    elementList = topDict[cName]
    print("\t\t %i topology:" % cName)
    # Print information about the n-th element in the m-th topology:
    n = 0
    el = elementList[n]
    print("\t\t %i-th element  = " % (n), el, end="")
    print("\n\t\t\twith final states =", el.tree.getFinalStates(), "\n\t\t\twith cross section =", el.weight, "\n\t\t\tand masses = ", el.mass)



 Decomposition Results: 
	  Total number of topologies: 1 
	  Total number of elements = 1 
		 111010100110101000 topology:
		 0-th element  =  (PV > gluino(1),gluino(2)), (gluino(1) > N1,q,q), (gluino(2) > N1,q,q)
			with final states = [N1, N1, q, q, q, q] 
			with cross section = ['1.30E+01 [TeV]:4.31E+00 [pb] (1000021, 1000021)', '8.00E+00 [TeV]:5.72E-01 [pb] (1000021, 1000021)'] 
			and masses =  [None, 6.75E+02 [GeV], 6.75E+02 [GeV], 2.00E+02 [GeV], 0.00E+00 [MeV], 0.00E+00 [MeV], 2.00E+02 [GeV], 0.00E+00 [MeV], 0.00E+00 [MeV]]


### Load the experimental results to be used.

In [5]:
listOfExpRes = database.getExpResults()

# Print basic information about the results loaded.
# Count the number of loaded UL and EM experimental results:
nUL, nEM = 0, 0
for exp in listOfExpRes:
    expType = exp.datasets[0].dataInfo.dataType
    if expType == 'upperLimit':
        nUL += 1
    elif expType == 'efficiencyMap':
        nEM += 1
print("\n Loaded Database with %i UL results and %i EM results " % (nUL, nEM))


 Loaded Database with 125 UL results and 60 EM results 


### Compute the theory predictions

In [6]:
print("\n Theory Predictions and Constraints:")
rmax = 0.
bestResult = None
allPredictions = []
for expResult in listOfExpRes:
    predictions = theoryPredictionsFor(expResult, topDict, combinedResults=False, marginalize=False)
    if not predictions:
        continue  # Skip if there are no constraints from this result
    print('\n %s ' % expResult.globalInfo.id)
    for theoryPrediction in predictions:
        dataset = theoryPrediction.dataset
        datasetID = theoryPrediction.dataId()
        mass = theoryPrediction.mass
        txnames = [str(txname) for txname in theoryPrediction.txnames]
        PIDs = theoryPrediction.PIDs
        print("------------------------")
        print("Dataset = ", datasetID)  # Analysis name
        print("TxNames = ", txnames)
        print("Prediction Mass = ", mass)  # Value for average cluster mass (average mass of the elements in cluster)
        print("Prediction PIDs = ", PIDs)  # Value for average cluster mass (average mass of the elements in cluster)
        print("Theory Prediction = ", theoryPrediction.xsection)  # Signal cross section
        print("Condition Violation = ", theoryPrediction.conditions)  # Condition violation values

        # Get the corresponding upper limit:
        print("UL for theory prediction = ", theoryPrediction.upperLimit)

        # Compute the r-value
        r = theoryPrediction.getRValue()
        print("r = %1.3E" % r)
        # Compute likelihoods for EM-type results:
        if dataset.getType() == 'efficiencyMap':
            theoryPrediction.computeStatistics()
            print('L_BSM, L_SM, L_max = %1.3E, %1.3E, %1.3E' % (theoryPrediction.likelihood(),
                  theoryPrediction.lsm(), theoryPrediction.lmax()))
        if r > rmax:
            rmax = r
            bestResult = expResult.globalInfo.id
        allPredictions.append(theoryPrediction)


 Theory Predictions and Constraints:

 ATLAS-SUSY-2015-06 
------------------------
Dataset =  SR5j
TxNames =  ['T1']
Prediction Mass =  [None, 6.75E+02 [GeV], 6.75E+02 [GeV], 0.00E+00 [MeV], 0.00E+00 [MeV], 2.00E+02 [GeV], 0.00E+00 [MeV], 0.00E+00 [MeV], 2.00E+02 [GeV]]
Prediction PIDs =  [[None, 1000021, 1000021, [-2, 1, 3, -1, -3, 2], [-2, 1, 3, -1, -3, 2], 1000022, [-2, 1, 3, -1, -3, 2], [-2, 1, 3, -1, -3, 2], 1000022]]
Theory Prediction =  1.30E+01 [TeV]:3.55E-02 [pb] (1000021, 1000021)
Condition Violation =  None
UL for theory prediction =  1.79E+00 [fb]
r = 1.981E+01
L_BSM, L_SM, L_max = 1.610E-44, 7.215E-03, 7.215E-03


NameError: name 'jet' is not defined